In [1]:
from util import *
import random
import tensorflow

# Prompt for mode
mode = "load"


# Set file names
file_train_instances = "train_stances.csv"
file_train_bodies = "train_bodies.csv"
file_test_instances = "test_stances_unlabeled.csv"
file_test_bodies = "test_bodies.csv"
file_predictions = 'predictions_test.csv'


# Initialise hyperparameters
r = random.Random()
lim_unigram = 5000
target_size = 4
hidden_size = 100
train_keep_prob = 0.6
l2_alpha = 0.00001
learn_rate = 0.01
clip_ratio = 5
batch_size_train = 500
epoch = 90


# Load data sets
raw_train = FNCData(file_train_instances, file_train_bodies)
raw_test = FNCData(file_test_instances, file_test_bodies)
n_train = len(raw_train.instances)


# Process data sets
train_set, train_stances, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer = \
    pipeline_train(raw_train, raw_test, lim_unigram=lim_unigram)
feature_size = len(train_set[0])
test_set = pipeline_test(raw_test, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer)




C:\Users\rushi\anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(len(train_set))

49972


In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(hidden_size, input_shape=(feature_size,), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(target_size, activation = 'softmax'))
model.add(tf.keras.layers.Dropout(0.1))

In [4]:
opt = tf.keras.optimizers.Adam(learning_rate=learn_rate)
scce = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=opt, loss=scce, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [5]:
x_val = train_set[-5000:]
y_val = train_stances[-5000:]
x_train = train_set[:-5000]
y_train = train_stances[:-5000]

In [6]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_val = np.asarray(x_val)
y_val = np.asarray(y_val)
test_set = np.asarray(test_set)

In [7]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size_train,
    epochs=epoch,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Train on 44972 samples, validate on 5000 samples
Epoch 1/90
44972/44972 [==============================] - 7s 147us/sample - loss: 1.6025 - sparse_categorical_accuracy: 0.7495 - val_loss: 0.3327 - val_sparse_categorical_accuracy: 0.9102
Epoch 2/90
44972/44972 [==============================] - 4s 78us/sample - loss: 0.7002 - sparse_categorical_accuracy: 0.8458 - val_loss: 0.2017 - val_sparse_categorical_accuracy: 0.9486
Epoch 3/90
44972/44972 [==============================] - 4s 78us/sample - loss: 0.4788 - sparse_categorical_accuracy: 0.8696 - val_loss: 0.1744 - val_sparse_categorical_accuracy: 0.9604
Epoch 4/90
44972/44972 [==============================] - 3s 66us/sample - loss: 0.4090 - sparse_categorical_accuracy: 0.8768 - val_loss: 0.1648 - val_sparse_categorical_accuracy: 0.9654 loss: 0.3993 - sp
Epoch 5/90
44972/44972 [==============================] - 3s 64us/sample - loss: 0.3546 - sparse_categorical_accuracy: 0.8837 - val_loss: 0.1331 - val_sparse

44972/44972 [==============================] - 3s 63us/sample - loss: 0.1640 - sparse_categorical_accuracy: 0.8975 - val_loss: 0.1034 - val_sparse_categorical_accuracy: 0.9890
Epoch 45/90
44972/44972 [==============================] - 3s 65us/sample - loss: 0.1664 - sparse_categorical_accuracy: 0.8990 - val_loss: 0.1105 - val_sparse_categorical_accuracy: 0.9890
Epoch 46/90
44972/44972 [==============================] - 3s 62us/sample - loss: 0.1649 - sparse_categorical_accuracy: 0.8989 - val_loss: 0.1066 - val_sparse_categorical_accuracy: 0.9892
Epoch 47/90
44972/44972 [==============================] - 3s 66us/sample - loss: 0.1693 - sparse_categorical_accuracy: 0.8955 - val_loss: 0.1130 - val_sparse_categorical_accuracy: 0.9896
Epoch 48/90
44972/44972 [==============================] - 3s 62us/sample - loss: 0.1711 - sparse_categorical_accuracy: 0.8960 - val_loss: 0.1095 - val_sparse_categorical_accuracy: 0.9884
Epoch 49/90
44972/44972 [==============================] - 3s 67us/sampl

Epoch 88/90
44972/44972 [==============================] - 3s 63us/sample - loss: 0.1634 - sparse_categorical_accuracy: 0.8990 - val_loss: 0.1252 - val_sparse_categorical_accuracy: 0.9898
Epoch 89/90
44972/44972 [==============================] - 3s 62us/sample - loss: 0.1601 - sparse_categorical_accuracy: 0.9010 - val_loss: 0.1264 - val_sparse_categorical_accuracy: 0.9898
Epoch 90/90
44972/44972 [==============================] - 3s 62us/sample - loss: 0.1568 - sparse_categorical_accuracy: 0.9003 - val_loss: 0.1310 - val_sparse_categorical_accuracy: 0.9882


In [8]:
model.save("./model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./model\assets


In [9]:
model_loaded = tf.keras.models.load_model('./model')

In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1000200   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
Total params: 1,000,604
Trainable params: 1,000,604
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
print(model_loaded.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1000200   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
Total params: 1,000,604
Trainable params: 1,000,604
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
def pred(predictions, i):
    result = np.argmax(predictions[i])
    result = label_ref_rev.get(result)
    return result

In [13]:
import numpy as np

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_val, y_val, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_set[:])

for i in range(len(predictions)):
    print("{} : {}".format(i + 1, pred(predictions, i)))

Evaluate on test data
5000/5000 [==============================] - 0s 96us/sample - loss: 0.1310 - sparse_categorical_accuracy: 0.9882
test loss, test acc: [0.1310089957821575, 0.9882]
1 : unrelated
2 : unrelated
3 : unrelated
4 : unrelated
5 : unrelated
6 : unrelated
7 : unrelated
8 : unrelated
9 : discuss
10 : unrelated
11 : discuss
12 : unrelated
13 : discuss
14 : unrelated
15 : discuss
16 : unrelated
17 : unrelated
18 : unrelated
19 : unrelated
20 : unrelated
21 : discuss
22 : discuss
23 : agree
24 : agree
25 : discuss
26 : unrelated
27 : unrelated
28 : discuss
29 : unrelated
30 : agree
31 : unrelated
32 : unrelated
33 : unrelated
34 : unrelated
35 : agree
36 : unrelated
37 : unrelated
38 : unrelated
39 : unrelated
40 : unrelated
41 : unrelated
42 : unrelated
43 : unrelated
44 : unrelated
45 : unrelated
46 : unrelated
47 : unrelated
48 : unrelated
49 : discuss
50 : discuss
51 : discuss
52 : unrelated
53 : unrelated
54 : agree
55 : unrelated
56 : discuss
57 : unrelated
58 : discuss


1003 : unrelated
1004 : discuss
1005 : unrelated
1006 : discuss
1007 : unrelated
1008 : unrelated
1009 : unrelated
1010 : unrelated
1011 : unrelated
1012 : unrelated
1013 : unrelated
1014 : unrelated
1015 : unrelated
1016 : unrelated
1017 : unrelated
1018 : unrelated
1019 : unrelated
1020 : unrelated
1021 : unrelated
1022 : unrelated
1023 : unrelated
1024 : unrelated
1025 : discuss
1026 : unrelated
1027 : unrelated
1028 : unrelated
1029 : unrelated
1030 : unrelated
1031 : discuss
1032 : unrelated
1033 : unrelated
1034 : discuss
1035 : unrelated
1036 : unrelated
1037 : unrelated
1038 : unrelated
1039 : agree
1040 : unrelated
1041 : agree
1042 : agree
1043 : unrelated
1044 : unrelated
1045 : unrelated
1046 : discuss
1047 : unrelated
1048 : unrelated
1049 : unrelated
1050 : agree
1051 : unrelated
1052 : agree
1053 : unrelated
1054 : unrelated
1055 : discuss
1056 : unrelated
1057 : agree
1058 : unrelated
1059 : unrelated
1060 : agree
1061 : unrelated
1062 : unrelated
1063 : discuss
1064 : 

2221 : discuss
2222 : unrelated
2223 : unrelated
2224 : unrelated
2225 : unrelated
2226 : unrelated
2227 : unrelated
2228 : unrelated
2229 : unrelated
2230 : unrelated
2231 : unrelated
2232 : unrelated
2233 : unrelated
2234 : unrelated
2235 : unrelated
2236 : unrelated
2237 : unrelated
2238 : unrelated
2239 : unrelated
2240 : unrelated
2241 : unrelated
2242 : discuss
2243 : unrelated
2244 : unrelated
2245 : unrelated
2246 : unrelated
2247 : unrelated
2248 : unrelated
2249 : discuss
2250 : unrelated
2251 : unrelated
2252 : unrelated
2253 : unrelated
2254 : discuss
2255 : unrelated
2256 : discuss
2257 : unrelated
2258 : discuss
2259 : unrelated
2260 : unrelated
2261 : unrelated
2262 : unrelated
2263 : agree
2264 : disagree
2265 : unrelated
2266 : unrelated
2267 : unrelated
2268 : unrelated
2269 : unrelated
2270 : unrelated
2271 : unrelated
2272 : unrelated
2273 : agree
2274 : discuss
2275 : unrelated
2276 : unrelated
2277 : unrelated
2278 : discuss
2279 : unrelated
2280 : agree
2281 : un

3527 : unrelated
3528 : unrelated
3529 : unrelated
3530 : unrelated
3531 : unrelated
3532 : unrelated
3533 : unrelated
3534 : unrelated
3535 : unrelated
3536 : unrelated
3537 : unrelated
3538 : unrelated
3539 : unrelated
3540 : unrelated
3541 : unrelated
3542 : unrelated
3543 : discuss
3544 : unrelated
3545 : unrelated
3546 : unrelated
3547 : discuss
3548 : unrelated
3549 : unrelated
3550 : unrelated
3551 : unrelated
3552 : unrelated
3553 : unrelated
3554 : unrelated
3555 : unrelated
3556 : discuss
3557 : unrelated
3558 : unrelated
3559 : discuss
3560 : unrelated
3561 : unrelated
3562 : unrelated
3563 : unrelated
3564 : unrelated
3565 : unrelated
3566 : unrelated
3567 : unrelated
3568 : discuss
3569 : unrelated
3570 : unrelated
3571 : discuss
3572 : unrelated
3573 : unrelated
3574 : discuss
3575 : agree
3576 : unrelated
3577 : unrelated
3578 : unrelated
3579 : unrelated
3580 : unrelated
3581 : unrelated
3582 : discuss
3583 : agree
3584 : unrelated
3585 : unrelated
3586 : unrelated
3587

4716 : unrelated
4717 : unrelated
4718 : discuss
4719 : agree
4720 : discuss
4721 : unrelated
4722 : unrelated
4723 : unrelated
4724 : unrelated
4725 : discuss
4726 : unrelated
4727 : unrelated
4728 : agree
4729 : unrelated
4730 : unrelated
4731 : unrelated
4732 : unrelated
4733 : unrelated
4734 : unrelated
4735 : unrelated
4736 : unrelated
4737 : unrelated
4738 : unrelated
4739 : unrelated
4740 : unrelated
4741 : unrelated
4742 : discuss
4743 : unrelated
4744 : unrelated
4745 : unrelated
4746 : discuss
4747 : unrelated
4748 : discuss
4749 : discuss
4750 : discuss
4751 : unrelated
4752 : discuss
4753 : unrelated
4754 : discuss
4755 : unrelated
4756 : unrelated
4757 : discuss
4758 : unrelated
4759 : discuss
4760 : unrelated
4761 : unrelated
4762 : unrelated
4763 : unrelated
4764 : unrelated
4765 : unrelated
4766 : unrelated
4767 : unrelated
4768 : unrelated
4769 : agree
4770 : unrelated
4771 : discuss
4772 : unrelated
4773 : discuss
4774 : unrelated
4775 : unrelated
4776 : unrelated
477

...

21591 : discuss
21592 : unrelated
21593 : discuss
21594 : unrelated
21595 : discuss
21596 : unrelated
21597 : unrelated
21598 : unrelated
21599 : discuss
21600 : unrelated
21601 : unrelated
21602 : unrelated
21603 : unrelated
21604 : discuss
21605 : agree
21606 : discuss
21607 : unrelated
21608 : discuss
21609 : agree
21610 : unrelated
21611 : unrelated
21612 : unrelated
21613 : discuss
21614 : discuss
21615 : unrelated
21616 : unrelated
21617 : unrelated
21618 : unrelated
21619 : unrelated
21620 : unrelated
21621 : unrelated
21622 : unrelated
21623 : unrelated
21624 : unrelated
21625 : discuss
21626 : unrelated
21627 : unrelated
21628 : discuss
21629 : unrelated
21630 : agree
21631 : unrelated
21632 : unrelated
21633 : unrelated
21634 : unrelated
21635 : unrelated
21636 : unrelated
21637 : discuss
21638 : unrelated
21639 : unrelated
21640 : unrelated
21641 : unrelated
21642 : unrelated
21643 : agree
21644 : unrelated
21645 : unrelated
21646 : unrelated
21647 : unrelated
21648 : agree


22537 : unrelated
22538 : discuss
22539 : unrelated
22540 : unrelated
22541 : unrelated
22542 : unrelated
22543 : discuss
22544 : unrelated
22545 : unrelated
22546 : unrelated
22547 : unrelated
22548 : unrelated
22549 : agree
22550 : unrelated
22551 : unrelated
22552 : unrelated
22553 : unrelated
22554 : unrelated
22555 : discuss
22556 : unrelated
22557 : unrelated
22558 : unrelated
22559 : unrelated
22560 : discuss
22561 : unrelated
22562 : unrelated
22563 : agree
22564 : unrelated
22565 : unrelated
22566 : unrelated
22567 : unrelated
22568 : unrelated
22569 : agree
22570 : unrelated
22571 : unrelated
22572 : unrelated
22573 : unrelated
22574 : unrelated
22575 : unrelated
22576 : unrelated
22577 : unrelated
22578 : discuss
22579 : discuss
22580 : unrelated
22581 : discuss
22582 : agree
22583 : unrelated
22584 : unrelated
22585 : discuss
22586 : unrelated
22587 : unrelated
22588 : unrelated
22589 : discuss
22590 : unrelated
22591 : unrelated
22592 : unrelated
22593 : unrelated
22594 : 

23280 : discuss
23281 : unrelated
23282 : unrelated
23283 : unrelated
23284 : unrelated
23285 : unrelated
23286 : discuss
23287 : discuss
23288 : unrelated
23289 : unrelated
23290 : discuss
23291 : unrelated
23292 : unrelated
23293 : discuss
23294 : unrelated
23295 : agree
23296 : unrelated
23297 : unrelated
23298 : unrelated
23299 : agree
23300 : unrelated
23301 : unrelated
23302 : unrelated
23303 : unrelated
23304 : unrelated
23305 : unrelated
23306 : unrelated
23307 : unrelated
23308 : discuss
23309 : unrelated
23310 : discuss
23311 : unrelated
23312 : unrelated
23313 : disagree
23314 : discuss
23315 : unrelated
23316 : unrelated
23317 : discuss
23318 : unrelated
23319 : discuss
23320 : unrelated
23321 : unrelated
23322 : unrelated
23323 : unrelated
23324 : unrelated
23325 : unrelated
23326 : unrelated
23327 : unrelated
23328 : unrelated
23329 : unrelated
23330 : discuss
23331 : unrelated
23332 : unrelated
23333 : discuss
23334 : unrelated
23335 : unrelated
23336 : unrelated
23337 :

23960 : discuss
23961 : discuss
23962 : unrelated
23963 : discuss
23964 : unrelated
23965 : unrelated
23966 : unrelated
23967 : unrelated
23968 : unrelated
23969 : unrelated
23970 : unrelated
23971 : unrelated
23972 : unrelated
23973 : unrelated
23974 : unrelated
23975 : unrelated
23976 : discuss
23977 : discuss
23978 : unrelated
23979 : discuss
23980 : discuss
23981 : unrelated
23982 : unrelated
23983 : unrelated
23984 : unrelated
23985 : discuss
23986 : discuss
23987 : unrelated
23988 : unrelated
23989 : unrelated
23990 : unrelated
23991 : agree
23992 : unrelated
23993 : unrelated
23994 : unrelated
23995 : unrelated
23996 : unrelated
23997 : unrelated
23998 : discuss
23999 : unrelated
24000 : unrelated
24001 : unrelated
24002 : unrelated
24003 : unrelated
24004 : unrelated
24005 : unrelated
24006 : unrelated
24007 : unrelated
24008 : discuss
24009 : unrelated
24010 : unrelated
24011 : discuss
24012 : unrelated
24013 : unrelated
24014 : unrelated
24015 : discuss
24016 : unrelated
2401

24678 : agree
24679 : unrelated
24680 : unrelated
24681 : unrelated
24682 : unrelated
24683 : unrelated
24684 : discuss
24685 : unrelated
24686 : unrelated
24687 : unrelated
24688 : unrelated
24689 : unrelated
24690 : unrelated
24691 : unrelated
24692 : discuss
24693 : discuss
24694 : discuss
24695 : unrelated
24696 : unrelated
24697 : unrelated
24698 : unrelated
24699 : unrelated
24700 : unrelated
24701 : unrelated
24702 : unrelated
24703 : unrelated
24704 : unrelated
24705 : unrelated
24706 : unrelated
24707 : discuss
24708 : discuss
24709 : unrelated
24710 : agree
24711 : unrelated
24712 : unrelated
24713 : unrelated
24714 : unrelated
24715 : unrelated
24716 : unrelated
24717 : discuss
24718 : unrelated
24719 : unrelated
24720 : unrelated
24721 : unrelated
24722 : unrelated
24723 : discuss
24724 : unrelated
24725 : unrelated
24726 : unrelated
24727 : agree
24728 : discuss
24729 : unrelated
24730 : unrelated
24731 : unrelated
24732 : unrelated
24733 : unrelated
24734 : unrelated
2473